In [3]:
from langchain_community.document_loaders import AzureAIDocumentIntelligenceLoader
import os
root_dir=os.getcwd()
file_name="test.pdf"
file_path = f"{root_dir}/app/data/{file_name}"
endpoint = "https://dn-aoai-korea-central.openai.azure.com/"
key = "EKa2fAi31qntjHBPS0gkMkkawq20j2obpI3syhMSeifQboaKztkcJQQJ99ALACNns7RXJ3w3AAABACOGmUzR"
# ClientAuthenticationError: (401) The DocumentModels_AnalyzeDocumentFromStream Operation under Azure AI Document Intelligence 2024-11-30 is not supported with the current subscription key and pricing tier OpenAI.S0.
# Code: 401
# Message: The DocumentModels_AnalyzeDocumentFromStream Operation under Azure AI Document Intelligence 2024-11-30 is not supported with the current subscription key and pricing tier OpenAI.S0.
loader = AzureAIDocumentIntelligenceLoader(
    api_endpoint=endpoint, api_key=key, file_path=file_path, api_model="prebuilt-layout"
)

documents = loader.load()
print(documents[0])

ClientAuthenticationError: (401) The DocumentModels_AnalyzeDocumentFromStream Operation under Azure AI Document Intelligence 2024-11-30 is not supported with the current subscription key and pricing tier OpenAI.S0.
Code: 401
Message: The DocumentModels_AnalyzeDocumentFromStream Operation under Azure AI Document Intelligence 2024-11-30 is not supported with the current subscription key and pricing tier OpenAI.S0.

In [1]:
import os
print(os.getcwd())


/Users/user/Documents/98.project/service-platform


In [1]:
from langchain_openai import AzureOpenAIEmbeddings

embeddings = AzureOpenAIEmbeddings(
    azure_endpoint="https://dn-aoai-east-us.openai.azure.com/",
    deployment="DN-text-embedding-3-small",
    openai_api_key="BXCs988ooSntF5CbLixyOeqUdtlNXduu14T2Q1mFxqNf9WCU12JPJQQJ99BAACYeBjFXJ3w3AAABACOGXw40"
)

input_text = "The meaning of life is 42"
vector = embeddings.embed_query(input_text)
print(vector[:3])

[0.020976193249225616, -0.02854049764573574, 0.003906212281435728]


In [2]:
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import AzureOpenAIEmbeddings
import os
file_path = f"{os.getcwd()}/app/data/test.pdf"
documents=PyMuPDFLoader(file_path).load()
splitter = RecursiveCharacterTextSplitter(chunk_size=30, chunk_overlap=10)
splits = splitter.split_documents(documents)
print(len(splits))
for doc in splits:
    print(doc.metadata)

embeddings = AzureOpenAIEmbeddings(
    azure_endpoint="https://dn-aoai-east-us.openai.azure.com/",
    deployment="DN-text-embedding-3-small",
    openai_api_key="BXCs988ooSntF5CbLixyOeqUdtlNXduu14T2Q1mFxqNf9WCU12JPJQQJ99BAACYeBjFXJ3w3AAABACOGXw40"
)
vectorstore = FAISS.from_documents(splits,embedding = embeddings)
# 파일 업로드시 vectorstore를 로컬에 저장
vectorstore_path=f"{os.getcwd()}/app/data"
vectorstore.save_local(vectorstore_path)


7
{'producer': 'PDFium', 'creator': 'PDFium', 'creationdate': 'D:20240826080442', 'source': '/Users/user/Documents/98.project/service-platform/app/data/test.pdf', 'file_path': '/Users/user/Documents/98.project/service-platform/app/data/test.pdf', 'total_pages': 1, 'format': 'PDF 1.7', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'moddate': '', 'trapped': '', 'page': 0}
{'producer': 'PDFium', 'creator': 'PDFium', 'creationdate': 'D:20240826080442', 'source': '/Users/user/Documents/98.project/service-platform/app/data/test.pdf', 'file_path': '/Users/user/Documents/98.project/service-platform/app/data/test.pdf', 'total_pages': 1, 'format': 'PDF 1.7', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'moddate': '', 'trapped': '', 'page': 0}
{'producer': 'PDFium', 'creator': 'PDFium', 'creationdate': 'D:20240826080442', 'source': '/Users/user/Documents/98.project/service-platform/app/data/test.pdf', 'file_path': '/Users/user/Documents/98.project/service-platform/app/dat

In [ ]:
# 채팅시 retriever시 vertostore 호출
vectorstore_path=f"{os.getcwd()}/app/data"
vectorstore=FAISS.load_local(vectorstore_path, embeddings,allow_dangerous_deserialization=True)
query = '장준혁'
docs = vectorstore.similarity_search(query)
for doc in docs:
    print(doc)


page_content='성    명 :  장준혁' metadata={'producer': 'PDFium', 'creator': 'PDFium', 'creationdate': 'D:20240826080442', 'source': '/Users/user/Documents/98.project/service-platform/app/data/test.pdf', 'file_path': '/Users/user/Documents/98.project/service-platform/app/data/test.pdf', 'total_pages': 1, 'format': 'PDF 1.7', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'moddate': '', 'trapped': '', 'page': 0}
1
page_content='과 정 명 :  1존 생활안전' metadata={'producer': 'PDFium', 'creator': 'PDFium', 'creationdate': 'D:20240826080442', 'source': '/Users/user/Documents/98.project/service-platform/app/data/test.pdf', 'file_path': '/Users/user/Documents/98.project/service-platform/app/data/test.pdf', 'total_pages': 1, 'format': 'PDF 1.7', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'moddate': '', 'trapped': '', 'page': 0}
1
page_content='귀하는 경기도국민안전체험관 1존 생활안전' metadata={'producer': 'PDFium', 'creator': 'PDFium', 'creationdate': 'D:20240826080442', 'source': '/Users/user/D

In [ ]:
from langchain.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            Answer the question using ONLY the following context. If you don't know the answer just say you don't know. DON'T make anything up.
            
            Context: {context}
            """,
        ),
        ("human", "{question}"),
    ]
)

def extract_metadata(docs_with_metadata):
    metadata = [doc_metadata for doc, doc_metadata in docs_with_metadata]
    return metadata


def format_docs(docs):
    return "\n\n".join(document.page_content for document in docs)

class MetaDataRetriever:
    def __init__(self, retriever):
        self.retriever = retriever
        self.metadata = None

    def __call__(self, *args, **kwargs):
        result = self.retriever(*args, **kwargs)
        self.metadata = extract_metadata(result)  
        return result

    def get_metadata(self):
        return self.metadata


vectorstore_path=f"{os.getcwd()}/app/data"
vectorstore=FAISS.load_local(vectorstore_path, embeddings,allow_dangerous_deserialization=True)
metadata_retriever = MetaDataRetriever(vectorstore.as_retriever())


from langchain.schema.runnable import RunnableLambda, RunnablePassthrough
from langchain_openai import AzureOpenAIEmbeddings, AzureChatOpenAI
message=" 성명 알려줘"
chain = (
    {
        "context": metadata_retriever | RunnableLambda(format_docs),
        "question": RunnablePassthrough(),
    }
    | prompt
    | AzureChatOpenAI(
        api_version=os.getenv("API_VERSION"),
        azure_endpoint=os.getenv("ENDPOINT"),
        azure_deployment=os.getenv("CHAT_MODEL"),
        api_key=os.getenv("API_KEY"),
        streaming=True,
    )
)
response = chain.invoke(message)
print(response)
metadata = metadata_retriever.get_metadata()
print(metadata)
